In [17]:
!cp "/content/drive/MyDrive/Colab Notebooks/Hac2021/best_data1.zip" "/content/"
import zipfile
z = zipfile.ZipFile("./best_data1.zip")
z.extractall("./")

In [18]:
import pandas as pd 
import numpy as np 
import datetime as dt
import math
from datetime import datetime
from collections import Counter

In [19]:
data=pd.read_csv("best_data1.csv").dropna().drop(["ship","Unnamed: 0"],axis=1)

In [24]:
data

,record,latitude,longitude,course,velocity,sure_tral,time_in_sec,distance,delta_time,linear_speed
0,1,-0.000009,-0.000048,269,0.0,0,1821,0.000000,1821,0.000000
1,1,0.000021,0.000012,245,0.0,0,5381,5.466741,3560,0.001536
2,1,0.000001,0.000002,288,0.0,0,8780,2.684734,3399,0.000790
3,1,-0.000009,-0.000018,207,0.0,0,13751,0.231358,4971,0.000047
4,1,0.000041,0.000002,112,0.0,0,16170,2.258584,2419,0.000934
...,...,...,...,...,...,...,...,...,...,...
4064014,34016,-0.016272,-0.016575,237,1.8,1,36030,5099.340099,3300,1.545255
4064015,34016,-0.012402,-0.037935,29,0.6,1,39939,2577.213083,3909,0.659302
4064016,34016,-0.012662,-0.049995,26,0.5,1,43520,4440.036232,3581,1.239887
4064017,34016,-0.010222,-0.038395,85,8.4,1,47130,5738.873985,3610,1.589716


In [5]:
points=dict(zip(data.record,data.sure_tral))

In [49]:
list(points.values()).count(0)

27054

In [50]:
list(points.values()).count(1)

1884

In [51]:
days=list(set(np.array(data.record)))
data_tensors=[]
for day in days:
  data_tensors+=[np.array(data[data.record==day].drop(['record'], axis = 1),"float64")]

AttributeError: ignored

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
data[['course', 'velocity']] = data[['course', 'velocity']].fillna(value=0)
data[['course', 'velocity']] = data[['course', 'velocity']].replace(np.nan, 0)
data[['course', 'velocity']] = data[['course', 'velocity']].replace('None', 0)

In [54]:
len(data[data["sure_tral"]==1].drop(["sure_tral"],axis=1))

67309

In [55]:
len(data[data["sure_tral"]==0].drop(["sure_tral"],axis=1))

3996710

In [60]:
df

,latitude,longitude,course,velocity,time_in_sec,distance,delta_time,linear_speed
0,-0.000009,-0.000048,269,0.0,1821,0.000000,1821,0.000000
1,0.000021,0.000012,245,0.0,5381,5.466741,3560,0.001536
2,0.000001,0.000002,288,0.0,8780,2.684734,3399,0.000790
3,-0.000009,-0.000018,207,0.0,13751,0.231358,4971,0.000047
4,0.000041,0.000002,112,0.0,16170,2.258584,2419,0.000934
...,...,...,...,...,...,...,...,...
19995,-0.057125,-0.085850,43,2.3,29014,15876.146231,3441,4.613818
19996,-0.056035,-0.084230,43,2.3,29155,11455.584403,141,81.245279
19997,-0.029105,-0.043490,45,2.4,32614,11238.681520,3459,3.249113
19998,-0.028005,-0.042040,44,2.3,32744,5813.349993,130,44.718077


In [61]:
df=pd.concat([data[data["sure_tral"]==0].drop(["sure_tral","record"],axis=1)[:10000],data[data["sure_tral"]==1].drop(["sure_tral","record"],axis=1)[:10000]],ignore_index=True)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df, [0]*10000+[1]*10000)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [74]:
list(clf.predict(data[data["sure_tral"]==0].drop(["sure_tral","record"],axis=1)[:1000000])).count(1)

34111

In [9]:
import pandas as pd 
import numpy as np 
import datetime as dt
from datetime import datetime
import geopy
from geopy import distance
pd.options.display.max_rows = 10000
import math


class functions_for_obrabotchic():
    ### дистанция между коорд
    def distance(row):
        coords_1 = (0,0)
        if row['ship'] == row['next_ship']:
            coords_2 = (row['next_lat'],row['next_lon'])
        else: 
            coords_2 = (0,0)
        distance = geopy.distance.geodesic(coords_1, coords_2).m
        return distance

    ### Время в секундах
    def time_to_sec(timestring):
      try:
        pt = datetime.strptime(timestring, '%H:%M:%S')
        total_seconds = pt.second + pt.minute*60 + pt.hour*3600
        return total_seconds
      except:
        return 0
    ### Средняя линейная скорость
    def linear_speed(row):
        if row['ship'] == row['next_ship']:
            if row['delta_time'] != 0:
                speed = row['distance'] / row['delta_time']
            else:
                speed = 0
        else:
                speed = 0
        return abs(speed) 
    def delta_time(row):
        if row['ship'] == row['next_ship'] or row['next_record'] == row['record']:
            delta = row['next_time'] - row['time_in_sec']
        else: 
            delta = row['time_in_sec']
        return abs(delta)

### Path до csv файла

def obrabotchic(path_to_csv):
    data = pd.read_csv(path_to_csv)
    data = data.dropna(subset=['ship', 'record','time'])
    data[['course', 'velocity']] = data[['course', 'velocity']].fillna(value=0)
    data[['course', 'velocity']] = data[['course', 'velocity']].replace(np.nan, 0)
    data[['course', 'velocity']] = data[['course', 'velocity']].replace('None', 0)
    data['time_in_sec'] =  data['time'].apply(lambda x: functions_for_obrabotchic.time_to_sec(x))
    data = data.sort_values(by=['ship', 'record', 'time_in_sec'])
    next_lat = data['latitude'].to_list()
    next_lon = data['longitude'].to_list()
    next_record = data['record'].to_list()
    next_record.pop(-1)
    next_lat.pop(-1)
    next_lon.pop(-1)
    next_record.insert(0, 1)
    next_lon.insert(0, 0)
    next_lat.insert(0, 0)
    data['next_record'] = next_record
    data['next_lat'] = next_lat
    data['next_lon'] = next_lon
    next_time = data['time_in_sec'].to_list()
    next_ship = data['ship'].to_list()
    next_time.pop(-1)
    next_ship.pop(-1)
    next_time.insert(0, 0)
    next_ship.insert(0, 1)
    data['next_time'] = next_time
    data['next_ship'] = next_ship
    data['distance'] = data.apply(lambda row: functions_for_obrabotchic.distance(row), axis=1)
    data['delta_time'] = data.apply(lambda row: functions_for_obrabotchic.delta_time(row), axis=1)
    data['linear_speed'] = data.apply(lambda row: functions_for_obrabotchic.linear_speed(row), axis=1)
    del data['next_lat']
    del data['next_lon']
    del data['next_ship']
    del data['next_time']
    del data['time']
    del data['next_record']
    new_angles=[0]*len(data.index)
    for i,ind in enumerate(data.index):
      base_angle = math.atan(data["latitude"][ind]/data["longitude"][ind])*57.2958
      bow_angle=int(data["course"][ind])
      new_angles[i] = (bow_angle-base_angle)%360
    data.assign(angle=new_angles)
    data['latitude'] = pd.to_numeric(data['latitude'] )
    data['longitude'] = pd.to_numeric(data['longitude'] )
    data.to_csv('new_best_data.csv',index=False)
    return data

In [11]:
data=obrabotchic("/content/control.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: RuntimeWarning: divide by zero encountered in double_scalars


In [79]:
with open('/content/RF.pd', 'wb') as f:
    pickle.dump(clf, f)


In [12]:
with open('/content/RF.pd', 'rb') as f:
    rf = pickle.load(f)

In [15]:
del data["ship"]

In [21]:
records=sorted(list(set(data.record)))

In [26]:
records[:10]

[28897, 28898, 28899, 28900, 28901, 28902, 28903, 28904, 28905, 28906]

In [18]:
data.to_csv("control_final.csv",index=False)

In [29]:
dfs=dict(zip(records,[data[data["record"]==i].drop(["record"],axis=1) for i in records]))

In [33]:
pvreds = dict(zip(records,[rf.predict(dfs[i]) for i in records]))

In [35]:
for i in records:
  pvreds[i]= list(pvreds[i]).count(1)/len(pvreds[i])

In [40]:
pred_diskr=dict(zip(records,[1 if pvreds[i]>0.75 else 0 for i in pvreds.keys()]))

In [48]:
file = open("test.txt", "w")
for i in pred_diskr:
  file.write(str(i)+" "+str(pred_diskr[i])+"\n")
file.close()

In [42]:
list(pred_diskr.values()).count(0)

4448

In [43]:
list(pred_diskr.values()).count(1)

672

In [87]:
list(preds).count(0)

16475

In [69]:
list(clf.predict(data[data["sure_tral"]==1].drop(["sure_tral","record"],axis=1)[:50000])).count(0)

16475

In [70]:
Precision=(50000-16475)/((50000-16475)+16475)
Recall=(50000-16475)/((50000-16475)+2309)

In [72]:
2*(Precision*Recall/(Precision+Recall))

0.7811589812894658

In [ ]:
len(data_tensors[0][1])

In [ ]:
print(set(map(len,data_tensors)))

In [ ]:
len(data_tensors[2][1])